<a href="https://colab.research.google.com/github/ahmedbouzid/Syteme-de-recomandation---texte-classification---arabe/blob/master/sysrec_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Syeteme de recomandation -Texte classification -Texte Mining- Machine learning**

**importation de librairie necessaire**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

**Charger le DataSet**

In [ ]:
citations = pd.read_excel('/content/citations (3).xlsx')
#tranformer le type de fichier csv


In [ ]:
citations.head()

,Texte
0,هو محمد بن عبد الله بن عبد المطّلب بن هاشم بن ...
1,وهو من ولد إسماعيل بن إبراهيم عليه السلام وهو ...
2,وقد ربي عليه السلام بلا أب لأن أباه عبد الله ق...
3,اللهم صلى على سيدنا محمد وعلى آله محمد
4,اللَّهُمَّ صَلِّ عَلَى سَيِّدِنَا مُحَمَّدٍ وَ...


**Netoyage de DataSet**

In [ ]:
#supprimer les valeur null
citations.isnull().sum()
citations.dropna(how='any',inplace=True)

In [ ]:
#Supprimer les colonnes repeté
citations.duplicated().sum()
citations.drop_duplicates(inplace=True)

In [ ]:
citations.sample(15)

,Texte
118,نجز مهامك الصعبة أولًا؛ أما السهل منها فسوف يت...
16,علموا المهنة فإنه يوشك أن يحتاج أحدكم إلى مهنته
31,قال الله تعالى ﴿ أَوَلَمْ يَرَ الَّذِينَ كَفَر...
48,الرَّابعُ : عَن أبي هُرَيْرةَ رضي اللَّه عنه ع...
113,لَقَدْ مَنَّ ٱللَّهُ عَلَى ٱلْمُؤمِنِينَ إِذْ ...
34,إذا كان الجميع يتحرك للأمام فالنجاح كفيل بأن ي...
36,قبل كل شيء الإستعداد هو سر النجاح
4,اللَّهُمَّ صَلِّ عَلَى سَيِّدِنَا مُحَمَّدٍ وَ...
133,عوضنا يا الله عن كُل ألم سكن جوفنا ، وعن كل مو...
75,انثروا القمح على رؤوس الجبال؛ لكي لا يقال جاع ...


**Texte preprocessing - netoyage**

Dans le traitement du langage naturel (NLP), la plupart du texte et des documents contiennent de nombreux mots redondants pour la classification du texte, tels que les mots vides, les fautes d'orthographe, les argots, etc. Dans cette section, nous expliquons brièvement certaines techniques et méthodes pour le texte. nettoyage et prétraitement des documents texte. Dans de nombreux algorithmes tels que les méthodes d'apprentissage statistiques et probabilistes, le bruit et les fonctionnalités inutiles peuvent affecter négativement les performances globales. Ainsi, l'élimination de ces caractéristiques est extrêmement importante.

In [ ]:
#supprimer les extra caratéres
def remove_extra_char_from_words(text):
    text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
    return text
    citations["Texte"] = citations["Texte"].apply(lambda text: remove_extra_char_from_words(text))

In [ ]:
import string

In [ ]:
arabic_punctuations = '''ﷺ`÷×؛<>_()﴿﴾*&^%][ـ،/«:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuation= string.punctuation

In [ ]:
## Supprimer la ponctuation
import string
PUNCT_TO_REMOVE = arabic_punctuations + english_punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
citations["Texte"] = citations["Texte"].apply(lambda text: remove_punctuation(text))

In [ ]:
citations.sample(15)

,Texte
22,يَوْمَئِذٍ تُعْرَضُونَ لاَ تَخْفَىٰ مِنكُمْ خَ...
114,يَوْمَئِذٍ يَوَدُّ ٱلَّذِينَ كَفَرُواْ وَعَصَو...
56,وَإِذَا سَأَلَكَ عِبَادِي عَنِّي فَإِنِّي قَرِ...
39,لا يكفي أن يكون الواحد اسمه في البطاقة أحمد وأ...
123,ٱللَّهُ ٱلصَّمَدُ
33,العمل الذي لا يحقق غير أرباح مادية هو عمل ضعيف
69,فعل المستحيل أحد دروب المرح
1,وهو من ولد إسماعيل بن إبراهيم عليه السلام وهو ...
13,عليك بالصدق وإن قتلك
100,عن أبي موسى الأشعري رضي الله عنه قال قال رسول ...


**Supprimer les signes diacritiques**

In [ ]:
arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

In [ ]:
#fonction pour supprimer les mots diacritique
def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text
citations["Texte"]=citations["Texte"].apply(lambda text: remove_diacritics(text))

**Normalisation de texte**

In [ ]:
#fonction pour normaliser le texte
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [ ]:
citations["Texte"]=citations["Texte"].apply(lambda text: normalize_arabic(text))

In [ ]:
#fonction pour supprimer les mots répéter
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [ ]:
citations["Texte"]=citations["Texte"].apply(lambda text: remove_repeating_char(text))

In [ ]:
citations["Texte"].head(30)

0     هو محمد بن عبد اله بن عبد المطلب بن هاشم بن عب...
1     وهو من ولد اسماعيل بن ابراهيم عليه السلام وهو ...
2     وقد ربي عليه السلام بلا اب لان اباه عبد اله قد...
3                 الهم صلي علي سيدنا محمد وعلي اله محمد
4     الهم صل علي سيدنا محمد وازواجه وذريته كما صليت...
5     لهم بارك علي سيدنا محمد وعلي ال سيدنا محمد كما...
6                                            صباح الخير
7                                       اجمل صباح صباحك
8                                 صباح الشهد صباح العسل
9     صباح الخير يا اهل الخير جعل اله يومكم نور وبلغ...
10                          صباحكم من الجنه ايها الاحبه
11    عن ابن عمر رضي اله عنهما قال نهينا عن التكلف ر...
12    لا يعجبكم من الرجل طنطنته ولكن من ادي الامانه ...
13                                 عليك بالصدق وان قتلك
14                 اذا كان الشغل مجهده فان الفراغ مفسده
15           مكسبه فيها بعض الدناءه خير من مساءله الناس
16      علموا المهنه فانه يوشك ان يحتاج احدكم الي مهنته
17    عن ابي عمرو وقيل ابي عمره سفيان بن عبد اله

**Supprimer les mots répetés**

In [ ]:
#fonction pour supprimer les mots répéter
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [ ]:
citations["Texte"]=citations["Texte"].apply(lambda text: remove_repeating_char(text))

In [ ]:
citations["Texte"].head(30)

0     هو محمد بن عبد اله بن عبد المطلب بن هاشم بن عب...
1     وهو من ولد اسماعيل بن ابراهيم عليه السلام وهو ...
2     وقد ربي عليه السلام بلا اب لان اباه عبد اله قد...
3                 الهم صلي علي سيدنا محمد وعلي اله محمد
4     الهم صل علي سيدنا محمد وازواجه وذريته كما صليت...
5     لهم بارك علي سيدنا محمد وعلي ال سيدنا محمد كما...
6                                            صباح الخير
7                                       اجمل صباح صباحك
8                                 صباح الشهد صباح العسل
9     صباح الخير يا اهل الخير جعل اله يومكم نور وبلغ...
10                          صباحكم من الجنه ايها الاحبه
11    عن ابن عمر رضي اله عنهما قال نهينا عن التكلف ر...
12    لا يعجبكم من الرجل طنطنته ولكن من ادي الامانه ...
13                                 عليك بالصدق وان قتلك
14                 اذا كان الشغل مجهده فان الفراغ مفسده
15           مكسبه فيها بعض الدناءه خير من مساءله الناس
16      علموا المهنه فانه يوشك ان يحتاج احدكم الي مهنته
17    عن ابي عمرو وقيل ابي عمره سفيان بن عبد اله

**Tokanization du texte**

La tokenisation est le processus consistant à décomposer un flux de texte en mots, phrases, symboles ou tout autre élément significatif appelé jetons. L'objectif principal de cette étape est d'extraire des mots individuels dans une phrase. Parallèlement à la classification de texte, dans le text mining, il est nécessaire d'incorporer un analyseur dans le pipeline qui effectue la tokenisation des documents

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#tokenize
def tokenize_text(text):
    tokens = nltk.word_tokenize(text)
    return tokens
citations["Texte"] = citations["Texte"].apply(lambda text:tokenize_text(text))

In [ ]:
citations["Texte"].head(20)

0     [هو, محمد, بن, عبد, اله, بن, عبد, المطلب, بن, ...
1     [وهو, من, ولد, اسماعيل, بن, ابراهيم, عليه, الس...
2     [وقد, ربي, عليه, السلام, بلا, اب, لان, اباه, ع...
3        [الهم, صلي, علي, سيدنا, محمد, وعلي, اله, محمد]
4     [الهم, صل, علي, سيدنا, محمد, وازواجه, وذريته, ...
5     [لهم, بارك, علي, سيدنا, محمد, وعلي, ال, سيدنا,...
6                                         [صباح, الخير]
7                                   [اجمل, صباح, صباحك]
8                            [صباح, الشهد, صباح, العسل]
9     [صباح, الخير, يا, اهل, الخير, جعل, اله, يومكم,...
10                    [صباحكم, من, الجنه, ايها, الاحبه]
11    [عن, ابن, عمر, رضي, اله, عنهما, قال, نهينا, عن...
12    [لا, يعجبكم, من, الرجل, طنطنته, ولكن, من, ادي,...
13                            [عليك, بالصدق, وان, قتلك]
14         [اذا, كان, الشغل, مجهده, فان, الفراغ, مفسده]
15    [مكسبه, فيها, بعض, الدناءه, خير, من, مساءله, ا...
16    [علموا, المهنه, فانه, يوشك, ان, يحتاج, احدكم, ...
17    [عن, ابي, عمرو, وقيل, ابي, عمره, سفيان, بن

**Supprimer les StopWords**

La classification des textes et des documents sur les réseaux sociaux, tels que Twitter, Facebook, etc. est généralement affectée par la nature bruyante (abréviations, formes irrégulières) des corpus de texte.

In [ ]:
pip install stop-words

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=8c77a2cb2c2fb83ea1f57bc7748f2abcd0cfa2cbbfe3eff70ffb0b6ea614708e
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [ ]:
from stop_words import get_stop_words
get_stop_words('arabic')

['فى',
 'في',
 'كل',
 'لم',
 'لن',
 'له',
 'من',
 'هو',
 'هي',
 'قوة',
 'كما',
 'لها',
 'منذ',
 'وقد',
 'ولا',
 'نفسه',
 'لقاء',
 'مقابل',
 'هناك',
 'وقال',
 'وكان',
 'نهاية',
 'وقالت',
 'وكانت',
 'للامم',
 'فيه',
 'كلم',
 'لكن',
 'وفي',
 'وقف',
 'ولم',
 'ومن',
 'وهو',
 'وهي',
 'يوم',
 'فيها',
 'منها',
 'مليار',
 'لوكالة',
 'يكون',
 'يمكن',
 'مليون',
 'حيث',
 'اكد',
 'الا',
 'اما',
 'امس',
 'السابق',
 'التى',
 'التي',
 'اكثر',
 'ايار',
 'ايضا',
 'ثلاثة',
 'الذاتي',
 'الاخيرة',
 'الثاني',
 'الثانية',
 'الذى',
 'الذي',
 'الان',
 'امام',
 'ايام',
 'خلال',
 'حوالى',
 'الذين',
 'الاول',
 'الاولى',
 'بين',
 'ذلك',
 'دون',
 'حول',
 'حين',
 'الف',
 'الى',
 'انه',
 'اول',
 'ضمن',
 'انها',
 'جميع',
 'الماضي',
 'الوقت',
 'المقبل',
 'اليوم',
 'ـ',
 'ف',
 'و',
 'و6',
 'قد',
 'لا',
 'ما',
 'مع',
 'مساء',
 'هذا',
 'واحد',
 'واضاف',
 'واضافت',
 'فان',
 'قبل',
 'قال',
 'كان',
 'لدى',
 'نحو',
 'هذه',
 'وان',
 'واكد',
 'كانت',
 'واوضح',
 'مايو',
 'ب',
 'ا',
 'أ',
 '،',
 'عشر',
 'عدد',
 'عدة',
 'عشرة',
 '

In [ ]:
#fonction pour supprimer les stopWords
def remove_stopwords(text):
    stop_words = get_stop_words('arabic') +['عبد','اله','ال','ابن','ولكن','ابي','عنهما','علي','ايها','عنه','يا','لان','اباه','ايها','وعلي','فانه','وقيل','يقول','وعن','بعدما','الي','وله','شيء','ويا','لي','قل','قلت','ولد']
    token_list = []
    for word in text:
        if not word in stop_words:
            token_list.append(word)
    return token_list

In [ ]:
citations['Texte']=citations["Texte"].apply(lambda text: remove_stopwords(text))

In [ ]:
citations["Texte"].head(20)

0     [محمد, المطلب, هاشم, مناف, قصي, كلاب, مره, كعب...
1     [اسماعيل, ابراهيم, السلام, اسماعيل, ابراهيم, ا...
2     [ربي, السلام, بلا, اب, مات, حملت, امه, بشهرين,...
3                        [الهم, صلي, سيدنا, محمد, محمد]
4     [الهم, صل, سيدنا, محمد, وازواجه, وذريته, صليت,...
5     [لهم, بارك, سيدنا, محمد, سيدنا, محمد, باركت, س...
6                                               [الخير]
7                                         [اجمل, صباحك]
8                                        [الشهد, العسل]
9     [الخير, اهل, الخير, جعل, يومكم, نور, وبلغكم, ا...
10                              [صباحكم, الجنه, الاحبه]
11             [عمر, رضي, نهينا, التكلف, رواه, البخاري]
12    [يعجبكم, الرجل, طنطنته, ادي, الامانه, وكف, اعر...
13                                 [عليك, بالصدق, قتلك]
14                        [الشغل, مجهده, الفراغ, مفسده]
15                 [مكسبه, الدناءه, خير, مساءله, الناس]
16           [علموا, المهنه, يوشك, يحتاج, احدكم, مهنته]
17    [عمرو, عمره, سفيان, رضي, رسول, الاسلام, قو

**Trouver les tiges de mots**

In [ ]:
from nltk import word_tokenize
from snowballstemmer import stemmer
ar_stemmer = stemmer("arabic")

In [ ]:
def stem_words(text):
  
    tokens = [ar_stemmer.stemWord(word) for text in tokens]
    return tokens
    citations["Texte"] = citations["Texte"].apply(lambda text: stem_words(text))

**Limmatiser le Texte**

In [ ]:
pip install qalsadi


In [ ]:
def lemmatize_words(text):
    tokens = [qalsadi.lemmatizer.Lemmatizer(text, pos='v') for text in tokens]
    return tokens
    citations["Texte"] = citations["Texte"].apply(lambda text: lemmatize_words(text))

**detokenazition  de texte pour calculer TF-IDF**

In [ ]:
citations['Texte']=citations['Texte'].astype(str)

In [ ]:
citations["Texte"]= citations['Texte'].str.replace(',', '')


In [ ]:
citations["Texte"]=citations['Texte'].str.replace("'", ' ')


In [ ]:
citations

,Texte
0,[ محمد المطلب هاشم مناف قصي كلاب م...
1,[ اسماعيل ابراهيم السلام اسماعيل ابراه...
2,[ ربي السلام بلا اب مات حملت امه ...
3,[ الهم صلي سيدنا محمد محمد ]
4,[ الهم صل سيدنا محمد وازواجه وذريته ...
...,...
133,[ عوضنا الم سكن جوفنا موقف ابكانا ...
134,[ الهم اني اسالك فارج الهم كاشف ال...
135,[ عباس رضي رسول الكرب العظيم الحليم ...
136,[ رسول صلي وسلم الكرب العظيم الحليم ...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(citations['Texte'])
tfidf_matrix.shape

(956, 5598)

In [ ]:

tfidf.get_feature_names()[200:210]

['ارجوكم',
 'ارحم',
 'ارحمني',
 'ارحموا',
 'اردت',
 'اردها',
 'ارزق',
 'ارزقني',
 'ارسل',
 'ارسلناك']

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim[1]


array([0.        , 1.        , 0.13094326, 0.        , 0.14127302,
       0.16315986, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [ ]:
indices = pd.Series(citations.index, index=citations['Texte']).drop_duplicates()

In [ ]:
indices[:10]


Texte
[ محمد   المطلب   هاشم   مناف   قصي   كلاب   مره   كعب   لءي   غالب ]                                                                                                                                                                                                                                                                                                                                                         0
[ اسماعيل   ابراهيم   السلام   اسماعيل   ابراهيم   السلام   الفيل   اتي   ملك   الحبشه   لهدم   الكعبه ]                                                                                                                                                                                                                                                                                                                      1
[ ربي   السلام   بلا   اب   مات   حملت   امه   بشهرين   الصحيح   المواهب ]                                                                                        

In [ ]:
def get_recommendations(Texte, cosine_sim=cosine_sim):
    idx = indices[Texte]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    citation_indice = [i[0] for i in sim_scores]
    return (citations['Texte'].iloc[citation_indice] ,sim_scores)

In [ ]:
get_recommendations(420)


(482    [ رسول   صلي   واله   وصحبه   وسلم   زينوا   ا...
 469    [ رسول   صلي   واله   وصحبه   وسلم   حفت   الج...
 468    [ رسول   صلي   واله   وصحبه   وسلم   حب   الدن...
 472    [ رسول   صلي   واله   وصحبه   وسلم   خيركم   ت...
 478    [ رسول   صلي   واله   وصحبه   وسلم   ركعتا   ا...
 462    [ رسول   صلي   واله   وصحبه   وسلم   اذانين   ...
 471    [ رسول   صلي   واله   وصحبه   وسلم   خياركم   ...
 459    [ رسول   صلي   واله   وصحبه   وسلم   احب   الب...
 481    [ رسول   صلي   واله   وصحبه   وسلم   زوروا   ا...
 476    [ رسول   صلي   واله   وصحبه   وسلم   ذاق   طعم...
 Name: Texte, dtype: object,
 [(453, 0.3804582589758662),
  (440, 0.31512847068420696),
  (439, 0.28719978771886207),
  (443, 0.2825478765263438),
  (449, 0.2804537884267867),
  (433, 0.27670573287404077),
  (442, 0.27625765040160855),
  (431, 0.27336774958453436),
  (452, 0.2629308385043796),
  (447, 0.26236347635128837)])

## Le top recommendations utilisant CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

count_jobid = count_vectorizer.fit_transform((citations['Texte'])) #ajustement et transformation du vecteur
count_jobid

<126x1169 sparse matrix of type '<class 'numpy.int64'>'
	with 1505 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
user_count = count_vectorizer.transform(citations['Texte'])
cos_similarity_countv = map(lambda x: cosine_similarity(user_count, x),count_jobid)


In [ ]:
output2 = list(cos_similarity_countv)

In [ ]:
print (output2)

[array([[1.        ],
       [0.        ],
       [0.        ],
       [0.23904572],
       [0.1       ],
       [0.12761265],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.  

## Les top recommandations utilisant CountVectorizer

In [ ]:
#Construire une carte inversée des indices et le contenue de citation
indices = pd.Series(citations.index, index=citations['Texte']).drop_duplicates()

In [ ]:
# Fonction qui prend le texte du citation en entrée et produit la plupart des citations similaires
def get_recommendations2(Texte, cosine_sim=output2):
    # Obtenez l'index du citation qui correspond au texte
    idx = indices[Texte]

    # Obtenez les scores de similarité pairwsie de tous les citations avec ce texte
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Trier les citations en fonction des scores de similarité
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtenez les scores des 10 citations les plus similaires
    sim_scores = sim_scores[1:11]

    # Obtenir les index des citations
    citation_indice = [i[0] for i in sim_scores]
    

    # Retournez le top 10 des citations les plus similaires
    return citations['Texte'].iloc[citation_indice]

In [ ]:
get_recommendations2(45)

46     [ الثالث   مسعود   رضي   النبي   الهم   اني   ...
44     [ الرابع   طريف   عدي   حاتم   الطاءي   رضي   ...
97     [ كعب   مالك   رضي   رايت   رسول   ياكل   بثلا...
11       [ عمر   رضي   نهينا   التكلف   رواه   البخاري ]
18     [ هريره   رضي   رسول   صلي   وسلم   قاربوا   و...
67       [ عمر   رضي   نهينا   التكلف   رواه   البخاري ]
17     [ عمرو   عمره   سفيان   رضي   رسول   الاسلام  ...
102    [ هريره   رضي   النبي   يحب   العطاس   ويكره  ...
98     [ عباس   رضي   النبي   البركه   تنزل   وسط   ا...
93     [ هريره   رضي   رسول   ايه   المنافق   ثلاث   ...
Name: Texte, dtype: object

##Recommandation utilisant Spacy

In [ ]:
import spacy

In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 1.4 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180943 sha256=4718ebe11a684c42d53006c70bdc89ccfff04980d06ca1e2acdb17223bc0f68c
  Stored in directory: /tmp/pip-ephem-wheel-cache-anc10npd/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
import en_core_web_sm

In [ ]:
nlp = en_core_web_sm.load()


#### Transformez le texte des copies en documents spacy


In [ ]:
#%%time
list_docs = []
for i in range(len(citations)):
  citations['spacy_doc'] = list(nlp.pipe(citations.Texte))


In [ ]:
   list_docs.append((citations['spacy_doc'],i))


In [ ]:
print(len(list_docs))

1


In [ ]:
print(list_docs)

[(0      ([, محمد,   , المطلب,   , هاشم,   , مناف,   , ...
1      ([, اسماعيل,   , ابراهيم,   , السلام,   , اسما...
2      ([, ربي,   , السلام,   , بلا,   , اب,   , مات,...
3      ([, الهم,   , صلي,   , سيدنا,   , محمد,   , مح...
4      ([, الهم,   , صل,   , سيدنا,   , محمد,   , واز...
                             ...                        
133    ([, عوضنا,   , الم,   , سكن,   , جوفنا,   , مو...
134    ([, الهم,   , اني,   , اسالك,   , فارج,   , ال...
135    ([, عباس,   , رضي,   , رسول,   , الكرب,   , ال...
136    ([, رسول,   , صلي,   , وسلم,   , الكرب,   , ال...
137    ([, هل,   , قلنا,   , وحده,   , شريك,   , المل...
Name: spacy_doc, Length: 126, dtype: object, 125)]


In [ ]:
def calculateSimWithSpaCy(nlp, citations, citations_text, n=6):
    # Calculer la similarité en utilisant spaCy
    list_sim =[]
    doc1 =nlp.pipe(citations.Texte)
    for i in citations.index:
      try:
            doc2 = list_docs[i][0]
            score = doc1.similarity(doc2)
            list_sim.append((doc1, doc2, list_docs[i][1],score))
      except:
        continue

    return  list_sim   

In [ ]:
citations.Texte[30]

'[ تعالي   واله   خلق   دابه   ماء   سوره   النور ]'

In [ ]:
 %%time

 df3 = calculateSimWithSpaCy(nlp, citations, citations.Texte[30], n=6)

CPU times: user 413 µs, sys: 24 µs, total: 437 µs
Wall time: 448 µs


In [ ]:
df3

[]

In [ ]:


df_recom_spacy = pd.DataFrame(df3).head(10)

In [ ]:
print (df_recom_spacy)

Empty DataFrame
Columns: []
Index: []


In [ ]:
df_recom_spacy.reset_index(inplace=True)


In [ ]:
index_spacy = df_recom_spacy[2]
list_scores = df_recom_spacy[3]

KeyError: ignored

# Les meilleures recommandations utilisant Spacy


In [ ]:
def get_recommendation2(top, citations, scores):
  recommendation = pd.DataFrame(columns = ['Texte'])
  count = 0
  for i in top:
      recommendation.at[count, 'Texte'] = citations['Texte'][i]
      count += 1
  return recommendation

In [ ]:
get_recommendation2(index_spacy, citations  , list_scores)

# **Utilisant KNN**

In [ ]:
from sklearn.neighbors import NearestNeighbors
n_neighbors = 11
KNN = NearestNeighbors(n_neighbors, p=2)
KNN.fit(tfidf_matrix)
NNs = KNN.kneighbors(tfidf_matrix, return_distance=True) 

In [ ]:
NNs[0][0][1:]

array([1.27460712, 1.34982247, 1.35067675, 1.36635528, 1.36985525,
       1.37586476, 1.38099981, 1.41421356, 1.41421356, 1.41421356])

## Les top recommandations utilisant KNN

In [ ]:
def get_recommendation2(top, citations, scores):
  recommendation = pd.DataFrame(columns = ['Texte'])
  count = 0
  for i in top:
      recommendation.at[count, 'Texte'] = citations['Texte'][i]
      count += 1
  return recommendation

In [ ]:
top = NNs[1][0][1:]
index_score = NNs[0][0][1:]

get_recommendation2(top, citations, index_score)

,Texte
0,[ الهم صلي سيدنا محمد محمد ]
1,[ لهم بارك سيدنا محمد سيدنا محمد ب...
2,[ تكن متيقنا ]
3,[ الهم صل سيدنا محمد وازواجه وذريته ...
4,[ الهم افتح ابواب رحمتك ]
5,[ اولٰءك لهم نصيب كسبوا وٱله سريع ...
6,[ خلق ٱلسمٰوٰت وٱلارض وٱختلاف ٱليل و...
7,[ واصل التقدم لامام ونفتح ابوابا جدي...
8,[ فاتك خير فادركه ادرك فاسبقه ]
9,[ يكفي الواحد اسمه البطاقه احمد يضع ...


# Word2Vect

In [ ]:
import gensim

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
from gensim import models


In [ ]:
from gensim.models import Word2Vec


In [ ]:
#splitting the description into words

corpus = []
for words in citations['Texte']:
    corpus.append(words.split())

In [ ]:
corpus

[['[',
  'محمد',
  'المطلب',
  'هاشم',
  'مناف',
  'قصي',
  'كلاب',
  'مره',
  'كعب',
  'لءي',
  'غالب',
  ']'],
 ['[',
  'اسماعيل',
  'ابراهيم',
  'السلام',
  'اسماعيل',
  'ابراهيم',
  'السلام',
  'الفيل',
  'اتي',
  'ملك',
  'الحبشه',
  'لهدم',
  'الكعبه',
  ']'],
 ['[',
  'ربي',
  'السلام',
  'بلا',
  'اب',
  'مات',
  'حملت',
  'امه',
  'بشهرين',
  'الصحيح',
  'المواهب',
  ']'],
 ['[', 'الهم', 'صلي', 'سيدنا', 'محمد', 'محمد', ']'],
 ['[',
  'الهم',
  'صل',
  'سيدنا',
  'محمد',
  'وازواجه',
  'وذريته',
  'صليت',
  'سيدنا',
  'ابراهيم',
  'وبارك',
  'سيدنا',
  'محمد',
  'وازواجه',
  'وذريته',
  'باركت',
  'سيدنا',
  'ابراهيم',
  'انك',
  'حميد',
  'مجيد',
  ']'],
 ['[',
  'لهم',
  'بارك',
  'سيدنا',
  'محمد',
  'سيدنا',
  'محمد',
  'باركت',
  'سيدنا',
  'ابراهيم',
  'انك',
  'حميد',
  'مجيد',
  'الهم',
  'وترحم',
  'سيدنا',
  'محمد',
  'سيدنا',
  'محمد',
  'ترحمت',
  'سيدنا',
  'ابراهيم',
  'سيدنا',
  'ابراهيم',
  'انك',
  'حميد',
  'مجيدالهم',
  'وتحن',
  'سيدنا',
  'محمد',
  'سيدنا',

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.phrases import Phrases, Phraser
from matplotlib import pyplot

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Téléchargement du modèle Word2Vec pré-entraîné par Google
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'
google_word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)


# Entraîner notre corpus avec Google Pretrained Model

google_model = Word2Vec(size = 300, window=5, min_count = 2, workers = -1)
google_model.build_vocab(corpus)

#model.intersect_word2vec_format('./word2vec/GoogleNews-vectors-negative300.bin', lockf=1.0, binary=True)

google_model.intersect_word2vec_format(EMBEDDING_FILE, lockf=1.0, binary=True)

google_model.train(corpus, total_examples=google_model.corpus_count, epochs = 5)

--2021-08-02 12:06:57--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.237.253
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.237.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  39.6MB/s    in 43s     

2021-08-02 12:07:40 (36.7 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



(0, 0)

In [ ]:
def vectors(x):
    
    # Creating a list for storing the vectors ('Description' into vectors)
    global word_embeddings
    word_embeddings = []

    # Reading the each 'Description'
    for line in citations['Texte']:
        avgword2vec = None
        count = 0
        for word in line.split():
            if word in google_model.wv.vocab:
                count += 1
                if avgword2vec is None:
                    avgword2vec = google_model[word]
                else:
                    avgword2vec = avgword2vec + google_model[word]
                
        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
            word_embeddings.append(avgword2vec)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
from keras.layers import Embedding


In [ ]:
# Recommander le Top 5 des citations similaires
def recommendations4(citations):
    
    # Appel des vecteurs de fonction
    vectors(citations)
    
    # Trouver la similarité en cosinus pour les vecteurs
    cosine_similarities = cosine_similarity(word_embeddings, word_embeddings)


    # Prendre le lien du Texte le stocker dans une nouvelle dataframe appelée « citations»
    citations = citations['Texte']

    # Reverse mapping de l'index
    indices = pd.Series(citations.index, index = citations['Texte']).drop_duplicates()
         
    idx = indices[Texte]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    Texte_indices = [i[0] for i in sim_scores[1:]]

    sim_scores =  [i[1] for i in sim_scores[1:]]
    return citations.iloc[Texte_indices]
    

   

In [ ]:
recommendations4(1)

TypeError: ignored